# pymoo: Multi-objective Optimization in Python for TRAP
**A report on implementing NSGA-II and NSGA-III using pymoo for the Tactical resource allocation problem**.


A supplementary material for `Fotedar,S., Strömberg,A.-B., Almgren,T. Bi-objective optimization of the tactical allocation of job types to machines. Mathematical modelling, theoretical analysis, and numerical tests.`

- In this report we utilize pymoo to obtain approximations of efficient frontiers for one of the instance of the TRAP to explain the algorithm
- Pymoo is multi-objective optimization library that can be installed for Python
- For more details on pymoo please visit [pymoo](https://pymoo.org/index.html)
- This notebook (.ipnyb file) can be downloaded from  [@github/SunneyF/Notebook](https://github.com/SunneyF/TRAP/blob/main/pymoo_TRAP-report.ipynb) 

**Summary**:
The conclusion of the report is that when using NSGA-II/NSGA-III without providing any initial solution we are not able to find any feasible solution even after three hours of computations. Secondly, when we provide initial solutions still the resulting efficient frontier is not superior to our proposed approach. In this report we have only shown computations for one of the instance. 

## 1. Download data: Data files must be first downloaded from [@github/SunneyF](https://github.com/SunneyF/TRAP) 

Following files must be available in the working directory:

1. [Capacity.csv](https://github.com/SunneyF/TRAP/blob/main/Capacity.csv)
2. [Constants.csv](https://github.com/SunneyF/TRAP/blob/main/Constants.csv)
3. [Instances 1-10.zip](https://github.com/SunneyF/TRAP/blob/main/Instances_1_10.zip) (extracted files must be in current directory)
4. [Instances 11-20](https://github.com/SunneyF/TRAP/blob/main/Instances_11_20.zip) (extracted files must be in current directory)
5. [Instances 21-30](https://github.com/SunneyF/TRAP/blob/main/Instances_21_30.zip) (extracted files must be in current directory)

## 2. Store data 

In [82]:
"""
store data 
"""
from csv import reader
import math
"""
 - load data corresponding to instance
 - files Capacity.csv and Constants.csv are the same for all the instances
"""
def load_data(instance):
    T=16; threshold=0.7; Tau=3; gamma=4; Work_Center_ID= range(1,125)
    with open('Constants.csv', 'r') as read_obj:
        # pass the file object to reader() to get the reader object
        csv_reader = reader(read_obj)
        # Iterate over each row in the csv using reader object
        JobType=[]; FeasibleMachines={}; NotQualifiedMachines={}; switch=0
        # switch=1: JobType, switch=2: FeasibleMachines, 
        # switch=3: NotQualifiedMachines
        row_counter=0
        for row in csv_reader:
            if len(row)!=0:
                if row[0][0]=='#':
                    if row[0][1:]== 'JobTypes':
                        switch=1
                    elif row[0][1:]=='FeasibleMachines':
                        switch=2
                    else:
                        switch=3
                else:
                    if switch==1:
                        JobType = [int(r) for r in row]
                    elif switch==2:
                        FeasibleMachines[int(row[0])] = \
                            [int(r) for r in row[1:] ] 
                    else:
                        NotQualifiedMachines[int(row[0])]= \
                           [int(r) for r in row[1:] ]         
    """
    Import capacity 
    """

    with open('Capacity.csv', 'r') as read_obj:
        # pass the file object to reader() to get the reader object
        csv_reader = reader(read_obj)
        # Iterate over each row in the csv using reader object
        Capacity={}

        for row in csv_reader:
            """
            Capcity[k,t] = ?
            """

            if len(row)!=0:
                 Capacity[int(row[0]), int(row[1]) ] = float(row[2])
    """
    Other parameters
    """
    with open('{}.csv'.format(instance), 'r') as read_obj:
    # pass the file object to reader() to get the reader object
        csv_reader = reader(read_obj)
        # Iterate over each row in the csv using reader object
        Pjk={};Demand={}; QualCost={}; switch=0
        # switch=1: JobType, switch=2: FeasibleMachines, \
        # switch=3: NotQualifiedMachines
        row_counter=0
        for row in csv_reader:
            if len(row)!=0:
                if row[0][0]=='#':
                    if row[0][1:3]== 'Pr':
                        switch=1
                    elif row[0][1:3]=='De':
                        switch=2
                    else:
                        switch=3
                else:
                    if switch==1:
                        Pjk[int(row[0]), int(row[1])] = float(row[2])
                    elif switch==2:
                        Demand[int(row[0]), int(row[1])] = float(row[2])
                    else:
                        QualCost[int(row[0]), int(row[1])]= float(row[2])       
     
    
    # x variable i.e. the no. of orders of job j assigned to machine k in time period t
    x = { kk:0 for kk in [(j,k,t) for j in JobType for k in FeasibleMachines[j] \
                          for t in range(1,T+1) if (j,t) in Demand.keys()]}

    len_x = len(x)

    total_length = len_x
    ID={}
    counters=0
    Id={}
    for kk in x.keys():    
        Id[kk]=counters
        counters+=1
    
    """
    Set upper bounds on variables
    """
    upp_bound =[]
    for k1 in x.keys():
       min_two = min(Demand[k1[0],k1[2]], math.floor(Capacity[k1[1], k1[2]]/Pjk[k1[0], k1[1]]))
       upp_bound.append(min_two)

    
    # Demand
    n_constraints=0
    for (j,t) in [(j,t) for j in JobType for t in range(1,T+1) if (j,t) in Demand.keys()]:
        n_constraints+=1 
        
    # Tau constraints
    for (j,t) in [(j,t) for j in JobType for t in range(1,T+1) if (j,t) in Demand.keys()]:
        n_constraints+=1 

            
    # qualification limit
    for t in range(1,T+1):
         n_constraints+=1
    for t in range(1,T+1):
            n_constraints+=1


    return JobType, FeasibleMachines, NotQualifiedMachines, Capacity, Pjk,Demand, \
           QualCost,total_length, Id, upp_bound, n_constraints,x

In [83]:
import os
files = [f for f in os.listdir('.') if os.path.isfile(f)]
instance='1y'
file_name= '{}.csv'.format(instance)
assert file_name in files, "instance {} not in the working directory".format(instance)
T=16; threshold=0.7; Tau=3; gamma=4; Work_Center_ID= range(1,125)
JobType, FeasibleMachines, NotQualifiedMachines, Capacity, Pjk,Demand, QualCost,\
total_length, Id, upp_bound, n_constraints,xs = load_data(instance)

### 3. Options within Pymoo

Next we use pymoo to identify approximations of efficient frontier. After a mathematical model is defined following steps are required before we use pymoo:

- **Implement the problem**: We need to set-up the problem in pymoo appropriately
- **Initialize an algorithm**: We need to define which multi-objective algorithm we want to use. Various options available can be seen in [algorithms](https://pymoo.org/algorithms/index.html). Some of the parameters that can be adjusted are population size and number of offsprings, and various other parameters as well. Parameters for NSGA-II can be seen in [NSGA-II](https://pymoo.org/algorithms/moo/nsga2.html).
- **Operators**: This is crucial in defining well-known genetic algorithm which is the basis of various multi-objective in-exact approaches as well. Within operators we have sampling (initial solutions), selection (module defining the mating selection during the execution of a genetic algorithm), crossover, and mutation (See [Operators](https://pymoo.org/operators/index.html) for more details). Here we elaborate sampling as that is the first aspect we define
    - Unbiased sampling - In which no initial feasible solutions are provided. Initial population is generated using some standard sampling techniques, i.e. <code>Random Sampling (‘real_random’, ‘int_random’, ‘bin_random’)</code> and <code>Latin Hypercube Sampling (‘real_lhs’)</code>.  See [unbiased sampling](https://pymoo.org/operators/sampling.html) for more details
    - Biased sampling - It can be very helpful if expert knowledge already exists, and known solutions are provided. See details in [biased sampling](https://pymoo.org/customization/initialization.html)
- **Optimize**: After defining the problem and algorithm we optimize the problem which generates set of efficient solutions (or its approximations)

#### 3.1 Solution representation
The solution according to the model is a vector of variables $(\mathbf{x},\mathbf{s},\mathbf{z},\mathbf{o})$, however, $\mathbf{s},\mathbf{z},\mathbf{o}$ are only auxiliary variables which can be deduced from $\mathbf{x}$. Hence, we decide to use $\mathbf{x}$ variables as a one-dimensional array.

#### 3.2 Unbiased initialization

We first try using an unbiased initialization. In below mentioned example we are going to solve instance '1y' using NSGA-II with termination = get_termination("time", "03:00:00")</code> (i.e. termination after 3 hours). We have used following parameters
<code>pop_size=30,
    n_offsprings=10,
    sampling=get_sampling("int_random"),
    crossover=get_crossover("int_sbx", prob=1.0, eta=3.0),
    mutation=get_mutation("int_pm", eta=3.0)</code>
    
Next we define the problem

In [95]:
# define the problem
## TRAP

import numpy as np
from pymoo.core.problem import ElementwiseProblem
import gurobipy as grb
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.factory import get_problem, get_sampling, get_crossover, get_mutation,get_termination
from pymoo.optimize import minimize
from pymoo.core.evaluator import Evaluator
from pymoo.core.population import Population
from pymoo.optimize import minimize

class MyProblem(ElementwiseProblem):
    """
    Defining the problem i.e. the objective function and constraints
    """
    
    def __init__(self):
        super().__init__(n_var=total_length,
                         n_obj=2,
                         n_constr=n_constraints,
                         xl=np.array([0]*total_length),
                         xu=np.array(upp_bound))

    def _evaluate(self, x, out, *args, **kwargs):
        """
        Objective functions
        """
        T=16; threshold=0.7; Tau=3; gamma=4; Work_Center_ID= range(1,125)

        loading_above={}
        for (k,t) in [(k,t) for k in Work_Center_ID for t in range(1,T+1)]:
            loading_above[k,t] = max((1/Capacity[k,t])*sum(Pjk[j,k]*x[Id[j,k,t]] for j in JobType \
                                                           if (j,k,t) in xs.keys() ) - threshold , 0)            
        
        o={}
        for t in range(1,T+1):
            o[t] = max([loading_above[k,t] for k in Work_Center_ID])
        
        z_var ={}
        for j in JobType:
            if j in NotQualifiedMachines.keys():
                for k in NotQualifiedMachines[j]:
                    if sum(x[Id[j,k,t]] for t in range(1,T+1) if (j,k,t) in xs.keys()) >0:
                        z_var[j,k]=1
                    else:
                        z_var[j,k]=0
        
        s_var={}
        for (j,k,t) in xs.keys():
            if x[Id[j,k,t]] >0:
                s_var[j,k,t]=1
            else:
                s_var[j,k,t]=0
        f1 = sum(o[t] for t in range(1,T+1))
        f2 = sum(QualCost[j,k]* z_var[j,k] for (j,k) in z_var.keys())
        
        Tau=3
        threshold=0.7
        gamma=4
        Work_Center_ID= range(1,125)
        conts=[]
        
        """
        Adding Constraints (Demand, tau, gamma)
        """
        
        # Demand
        for (j,t) in [(j,t) for j in JobType for t in range(1,T+1) if (j,t) in Demand.keys()]:
            conts.append(Demand[j,t]-sum(x[Id[j,k,t]] for k in FeasibleMachines[j]) ) 
        
        # Tau constraints
        for (j,t) in [(j,t) for j in JobType for t in range(1,T+1) if (j,t) in Demand.keys()]:
            conts.append(sum(s_var[j,k,t] for k in FeasibleMachines[j] if (j,k,t) in xs.keys() )-Tau)
            
        # upper bound on o
        for t in range(1,T+1):
            conts.append(o[t]-1+threshold)
        
        ##############################################################
        # to impose gamma limitation we solve an LP
        ##############################################################
        
        model = grb.Model()
        vv = model.addVars(grb.tuplelist([(j,k,t) for j in JobType for k in FeasibleMachines[j] \
                                          for t in range(1,T+1) if (j,k) in z_var]), vtype=grb.GRB.BINARY,name='vv')
        maxi = model.addVars(grb.tuplelist([t for t in range(1,T+1) ]), vtype=grb.GRB.CONTINUOUS,name='vv')
        model.setObjective(sum(maxi[t] for t in range(1,T+1)),grb.GRB.MINIMIZE)
        for t in range(1,T+1):
            model.addConstr(grb.quicksum(vv[j,k,t] for (j,k) in z_var.keys()  )-gamma <= maxi[t])
        for (j,k) in z_var.keys():
                    for t in range(1,T+1):
                        if (j,k,t) in s_var.keys():
                            model.addConstr(grb.quicksum(vv[j,k,l] for l in range(1,t+1))>= s_var[j,k,t])
        model.setParam("OutputFlag",0)
        model.setParam("timelimit",100)
        model.optimize()
        
        ################################################################
        
        for t in range(1,T+1):
            if maxi[t].x > 0:
                conts.append(maxi[t].x)
            else:
                conts.append(0)
                                    
        out["F"] = [f1, f2]
        out["G"] = conts


        
problems = MyProblem()

In [85]:
print("Size of the solution vector i.e. x variables is {}".format(problems.n_var))

Size of the solution vector i.e. x variables is 494747


In [58]:
# now we define sampling, crossover, and mutation and run NSGA-II

from pymoo.factory import get_sampling, get_crossover, get_mutation
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.factory import get_termination
from pymoo.factory import get_sampling, get_crossover, get_mutation

algorithm = NSGA2(
    pop_size=30,
    n_offsprings=10,
    sampling=get_sampling("int_random"),
    crossover=get_crossover("int_sbx", prob=1.0, eta=3.0),
    mutation=get_mutation("int_pm", eta=3.0),
    eliminate_duplicates=True
)

# stop after 3 hours
termination = get_termination("time", "03:00:00")

from pymoo.optimize import minimize

res = minimize(problems,
               algorithm,
               termination,
               seed=1,
               save_history=True,
               verbose=True)

n_gen |  n_eval |   cv (min)   |   cv (avg)   |  n_nds  |     eps      |  indicator  
    1 |      30 |  4.38862E+05 |  4.39538E+05 |       1 |            - |            -
    2 |      40 |  4.38862E+05 |  4.39538E+05 |       1 |  0.00000E+00 |            f
    3 |      50 |  4.38862E+05 |  4.39536E+05 |       1 |  0.00000E+00 |            f
    4 |      60 |  4.38862E+05 |  4.39533E+05 |       1 |  0.00000E+00 |            f
    5 |      70 |  4.38862E+05 |  4.39533E+05 |       1 |  0.00000E+00 |            f
    6 |      80 |  4.38862E+05 |  4.39533E+05 |       1 |  0.00000E+00 |            f
    7 |      90 |  4.38862E+05 |  4.39533E+05 |       1 |  0.00000E+00 |            f
    8 |     100 |  4.38862E+05 |  4.39533E+05 |       1 |  0.00000E+00 |            f
    9 |     110 |  4.38862E+05 |  4.39533E+05 |       1 |  0.00000E+00 |            f
   10 |     120 |  4.38862E+05 |  4.39533E+05 |       1 |  0.00000E+00 |            f
   11 |     130 |  4.38862E+05 |  4.39533E+05 |       

As visible from above log no feasible solution was found. Note that <code>cv (min)</code> is the minumum constraint voilation and <code>cv (avg)</code> the average voilation. The computations were done for 3 hours. A total of 30 iterations were concluded in 3 hours. See [Logfile](https://pymoo.org/interface/display.html) for documentation on displyed items. One of the possible reasons that each iteration is slow is that for a given $\mathbf{x}$ computing $\mathbf{z}$ requires solving a LP (see Proposition 3 in the manuscript). Also length of the vector of variables is <code> problems.n_var=494747</code>. The structure of the problem with various auxilary variables also does not help. Hence, it might be helpful if we can initialize with a few feasible solutions such that we have a better chance of finding efficient solutions.

#### 3.2 Biased initialization
Based on the discouraging results from unbiased initialization. We consider providing a few feasible solutions to help NSGA-II to improve the solution quality. See [Biased initialization](https://pymoo.org/customization/initialization.html) for more details in Pymoo. Here instead of standard sampling techniques from Pymoo that are used above. We look to provide a population of feasible solutions that are part of the initialization. For this purpose we utilize a heuristic to find reasonable solutions to begin with. We have utilized the starting heuristic mentioned in the manuscript.

In [88]:
# solving an optimization problem corresponding to each time period (see Algorithm 1 in the manuscript)

def solve_t(t,NQualCost,Tau,threshold,gamma_temp, func, S_bar,loc):
    # Model
    # find 
    model_t = grb.Model("TRA_t")
    
    x_t= {} # integer variable for the number of orders
    s_t={}  # binary variable 
    z_t={} # qualification variable
    o_t={}  # resource imbalance
    
    # variables
    for (j,k) in [(a,b) for a in JobType for b in FeasibleMachines[a] if (a,t) in Demand.keys()]:
        x_t[j,k] = model_t.addVar(vtype= grb.GRB.INTEGER)
    for (j,k) in x_t:
        s_t[j,k] = model_t.addVar(vtype= grb.GRB.BINARY)
    for (j,k) in [(j,k) for j in JobType for k in FeasibleMachines[j]]:
            if j in NotQualifiedMachines.keys():
                if k in NotQualifiedMachines[j]:
                    z_t[j,k] = model_t.addVar(vtype= grb.GRB.BINARY)
    
    o_t = model_t.addVar(vtype=grb.GRB.CONTINUOUS,lb=0.0,ub= 1.0- threshold)
    if func==1:
      model_t.setObjective(o_t + .01*sum(QualCost[j,k]*z_t[j,k] for (j,k) in z_t.keys()),grb.GRB.MINIMIZE) 
    else:
      model_t.setObjective(sum(QualCost[j,k]*z_t[j,k] for (j,k) in z_t.keys())+o_t ,grb.GRB.MINIMIZE)
    # Constraints
    # 2.2 (Demand constraints)
    Assignments = {}
    for j in [j for j in JobType  if  (j,t) in Demand]:
        Assignments[j] = model_t.addConstr(grb.quicksum(x_t[j,k] for k in FeasibleMachines[j]) == Demand[j,t]) 
    # 2.3 (defining s variables)
    s_var = {}
    for (j,k) in s_t:
        s_var[j,k] = model_t.addConstr(Demand[j,t]*s_t[j,k] >= x_t[j,k])
    # 2.4 (routing constraints)
    route = {}
    for (j,t) in [(j,t) for j in JobType for t in range(1,T+1)]:
        if (j,t) in Demand:
            route[j,t] = model_t.addConstr(grb.quicksum(s_t[j,k] for k in FeasibleMachines[j] if (j,k) in s_t) <= Tau)           
    # 2.5 (min-max constraints)
    MinmaxCategory={}
    for k in Work_Center_ID: 
            MinmaxCategory[k] = model_t.addConstr((1/Capacity[k,t])*grb.quicksum(Pjk[j,k]*x_t[j,k] for j in JobType if (j,k) in x_t ) - threshold <= o_t)   
    
    model_t.addConstr(sum(z_t[j,k] for j in JobType for k in FeasibleMachines[j] if (j,k) in z_t.keys())  <= gamma_temp,\
                      name = "Budget")  
    
    temp={}
    for (j,k) in z_t:
            if NQualCost[j,k]!=0:
                if (j,k) in s_t:
                    temp[j,k] = model_t.addConstr(z_t[j,k] >= s_t[j,k]) 
            else:
                z_t[j,k]==0

    model_t.Params.OutputFlag=0
    model_t.optimize()
    
    status = model_t.status
    if status == grb.GRB.Status.UNBOUNDED:
        print('The model cannot be solved because it is unbounded')
        exit(0)
        
    elif status == grb.GRB.Status.INFEASIBLE or status == grb.GRB.Status.INF_OR_UNBD:
        print('The model is infeasible; computing IIS')
        model_t.computeIIS()
        print('\nThe following constraint(s) cannot be satisfied:')
        for c in model_t.getConstrs():
            if c.IISConstr:              
                print('%s' % c.constrName) 
    
    X={};S={};Z={}
    for (j,k) in x_t:
        X[j,k]= x_t[j,k].X
        S[j,k]=1 if x_t[j,k].X>0 else 0
    for (j,k) in z_t:
        Z[j,k]= z_t[j,k].X 
    obj = o_t.X
    return X,S,Z, obj

Now we find feasible solutions using the function <code>solve_t </code> based on Algorithm 1 from the manuscript.

In [89]:
#############################################################
import copy
import gurobipy as grb
gamma=4
S_bar={}
loc=0
for f in [1,2]:  # 1: prioritizing g_1 and less on g_2 #2: prioritizing g_1 and less on g_2
    S_0 = {};Z_0={};X_0={}
    obj_0=0
    NQualCost= copy.deepcopy(QualCost)
    for t in range(1,T+1):
        #print('Time=%s' %(str(t)))
        if t== 5:       
            gamma_t= 7
            X,S,Z, obj_t =solve_t(t,NQualCost,Tau,threshold,gamma_t,f, S_bar, loc)
        else:
            X,S,Z, obj_t =solve_t(t,NQualCost,Tau,threshold,gamma,f, S_bar, loc)

        for (j,k) in S:
            X_0[j,k,t]= X[j,k]
            S_0[j,k,t] = S[j,k]
        for (j,k) in Z:
            Z_0[j,k,t]=Z[j,k]
            if Z[j,k]==1:
                NQualCost[j,k] =0

        obj_0 = obj_0 + obj_t
    if f==1:
         S_T = copy.deepcopy(S_0);X_T=copy.deepcopy(X_0); obj_T= copy.deepcopy(obj_0)
    else:
        S_B = copy.deepcopy(S_0);X_B=copy.deepcopy(X_0); obj_B= copy.deepcopy(obj_0)

Academic license - for non-commercial use only - expires 2022-11-28
Using license file C:\Users\sunney\gurobi.lic


Now the two solutions with their s variables are stored in S_T and S_B. They are used to find corresponding $\mathbf{z}$ variables as in Prop. 3 from manuscript.

In [90]:
##############################################################
# to impose gamma limitation we solve a LP
##############################################################

def findZ(S):
    model = grb.Model()
    vv = model.addVars(grb.tuplelist([(j,k,t) for j in JobType for k in FeasibleMachines[j] for t in range(1,T+1)\
                                      if (j,k) in QualCost]), vtype=grb.GRB.BINARY,name='vv')
    model.setObjective(sum(QualCost[j,k]*vv[j,k,t] for (j,k,t) in vv.keys()), grb.GRB.MINIMIZE)
    for t in range(1,T+1):
        model.addConstr(grb.quicksum(vv[j,k,t] for (j,k) in QualCost.keys()  ) <= gamma)
    for (j,k) in QualCost:
                for t in range(1,T+1):
                    if (j,k,t) in S.keys():
                        if k in NotQualifiedMachines[j]:
                         model.addConstr(grb.quicksum(vv[j,k,l] for l in range(1,t+1))>= S[j,k,t])
    model.setParam("OutputFlag",0)
    model.setParam("timelimit",500)
    model.optimize()
    if model.status==2:
        Z = {(j,k,t): vv[j,k,t].x for (j,k,t) in vv.keys()}
    return Z

In [91]:
# Find corresponding z values

for ll in ['T', 'B']:
    if ll=='T':
        Z_T= findZ(S_T)
        o= copy.deepcopy(obj_T)
        g2_T=sum(QualCost[j,k]*Z_T[j,k,t] for (j,k,t) in Z_T.keys())
        qualifications_T = sum(Z_T[j,k,t] for (j,k,t) in Z_T.keys() if t==5)
        g1_T = o
    else:
        Z_B= findZ(S_B)
        o= copy.deepcopy(obj_B)
        g2_B=sum(QualCost[j,k]*Z_B[j,k,t] for (j,k,t) in Z_B.keys())
        qualifications_B = sum(Z_B[j,k,t] for (j,k,t) in Z_B.keys() if t==5)
        g1_B = o

In [92]:
X_Ta=[0]*len(X_T);X_Ba = [0]*len(X_T)
counters=0
Id={}
for (j,k,t) in [(j,k,t) for j in JobType for k in FeasibleMachines[j] for t in range(1,T+1) if (j,t) in Demand.keys()]:
    X_Ta[counters]=X_T[j,k,t]
    X_Ba[counters]=X_B[j,k,t]
    Id[j,k,t] = counters
    counters=counters+1

X=np.column_stack((X_Ta, X_Ba))

##### Now we run algorithm for one of the instances using the two solutions found above

In [27]:
        
problems = MyProblem()
pop = Population.new("X", np.transpose(X))
Evaluator().eval(problems, pop)
termination = get_termination("time", "04:00:00")


algorithm = NSGA2(
    n_offsprings=20,
    sampling=pop,
    crossover=get_crossover("int_sbx", prob=1.0, eta=3.0),
    mutation=get_mutation("int_pm", eta=3.0),
    eliminate_duplicates=True
)

res = minimize(problems,
               algorithm,
               termination,
               seed=1,
               save_history=True,
               verbose=True)

n_gen |  n_eval |   cv (min)   |   cv (avg)   |  n_nds  |     eps      |  indicator  
    1 |       0 |  0.00000E+00 |  0.00000E+00 |       2 |            - |            -
    2 |      20 |  0.00000E+00 |  8.22730E+03 |       4 |  0.00000E+00 |            f
    3 |      40 |  0.00000E+00 |  9.57330E+03 |       4 |  0.00000E+00 |            f
    4 |      60 |  0.00000E+00 |  1.00799E+04 |       4 |  0.00000E+00 |            f
    5 |      80 |  0.00000E+00 |  1.02396E+04 |       4 |  0.00000E+00 |            f
    6 |     100 |  0.00000E+00 |  1.04686E+04 |       4 |  0.00000E+00 |            f
    7 |     120 |  0.00000E+00 |  8.96838E+03 |       4 |  0.00000E+00 |            f
    8 |     140 |  0.00000E+00 |  7.83615E+03 |       4 |  0.00000E+00 |            f
    9 |     160 |  0.00000E+00 |  7.18228E+03 |       4 |  0.00000E+00 |            f
   10 |     180 |  0.00000E+00 |  6.68504E+03 |       4 |  0.00000E+00 |            f
   11 |     200 |  0.00000E+00 |  5.97709E+03 |       

In [29]:
print("No. of feasible solutions are {}".format(len(res.X)))

No. of feasible solutions are 48


A total of 48 different solutions are found. Now we check their corresponding objective values

In [30]:
res.F

array([[ 0.21203496, 11.        ],
       [ 2.8401633 ,  9.        ],
       [ 2.8401633 ,  9.        ],
       [ 2.8401633 ,  9.        ],
       [ 0.21203496, 11.        ],
       [ 0.21203496, 11.        ],
       [ 0.21203496, 11.        ],
       [ 0.21203496, 11.        ],
       [ 0.21203496, 11.        ],
       [ 0.21203496, 11.        ],
       [ 0.21203496, 11.        ],
       [ 0.21203496, 11.        ],
       [ 0.21203496, 11.        ],
       [ 0.21203496, 11.        ],
       [ 0.21203496, 11.        ],
       [ 0.21203496, 11.        ],
       [ 0.21203496, 11.        ],
       [ 0.21203496, 11.        ],
       [ 0.21203496, 11.        ],
       [ 0.21203496, 11.        ],
       [ 0.21203496, 11.        ],
       [ 0.21203496, 11.        ],
       [ 0.21203496, 11.        ],
       [ 0.21203496, 11.        ],
       [ 0.21203496, 11.        ],
       [ 0.21203496, 11.        ],
       [ 0.21203496, 11.        ],
       [ 0.21203496, 11.        ],
       [ 0.21203496,

We note that there are only two non-dominated points found and various alternate solutions corresponding to it. Now we use [hypervolume indicator](https://pymoo.org/misc/indicators.html) to assess performance of NSGA-II. We use reference point for $g_{1}$ as $T*(1-threshold)$ i.e. 4.8 and for $g_{2}$ as $gamma*T$ i.e. 64

In [49]:
from pymoo.factory import get_performance_indicator
unique=np.unique(res.F, axis=0)
hv = get_performance_indicator("hv", ref_point=np.array([T*(1-threshold), T*gamma]))
print("hyper volume =", hv.do(unique))
print("non-dominated points identified by NSGA-II=\n{}".format(unique))

hyper volume = 247.08182047413607
non-dominated points identified by NSGA-II=
[[ 0.21203496 11.        ]
 [ 2.8401633   9.        ]]


Now we can compare above results with the results from our proposed algorithm's results.

In [46]:
NDP = np.array([[ 0.0,13], [2.4855,9], [0.0288,11], [ 1.55,10]])
hv = get_performance_indicator("hv", ref_point=np.array([T*(1-threshold), T*gamma]))
print("hyper volume =", hv.do(NDP))


hyper volume = 259.90690000000006


Since the hypervolume of our results is of course greater than NSGA-II our proposed approach is much better. Infact both points found by NSGA-II are dominated by the  found by the points found by our algorithm. Similar experiments can be done for other instances as well.

# NSGA-III

Visit [NSGA-III](https://pymoo.org/algorithms/moo/nsga3.html) for details on this algorithm implementation in pymoo

In [99]:
from pymoo.algorithms.moo.nsga3 import NSGA3
from pymoo.factory import get_problem, get_reference_directions, get_termination, get_visualization
from pymoo.optimize import minimize
from pymoo.visualization.scatter import Scatter


In [101]:
class MyProblem(ElementwiseProblem):
    """
    Defining the problem i.e. the objective function and constraints
    """
    
    def __init__(self):
        super().__init__(n_var=total_length,
                         n_obj=2,
                         n_constr=n_constraints,
                         xl=np.array([0]*total_length),
                         xu=np.array(upp_bound))

    def _evaluate(self, x, out, *args, **kwargs):
        """
        Objective functions
        """
        T=16; threshold=0.7; Tau=3; gamma=4; Work_Center_ID= range(1,125)

        loading_above={}
        for (k,t) in [(k,t) for k in Work_Center_ID for t in range(1,T+1)]:
            loading_above[k,t] = max((1/Capacity[k,t])*sum(Pjk[j,k]*x[Id[j,k,t]] for j in JobType \
                                                           if (j,k,t) in xs.keys() ) - threshold , 0)            
        
        o={}
        for t in range(1,T+1):
            o[t] = max([loading_above[k,t] for k in Work_Center_ID])
        
        z_var ={}
        for j in JobType:
            if j in NotQualifiedMachines.keys():
                for k in NotQualifiedMachines[j]:
                    if sum(x[Id[j,k,t]] for t in range(1,T+1) if (j,k,t) in xs.keys()) >0:
                        z_var[j,k]=1
                    else:
                        z_var[j,k]=0
        
        s_var={}
        for (j,k,t) in xs.keys():
            if x[Id[j,k,t]] >0:
                s_var[j,k,t]=1
            else:
                s_var[j,k,t]=0
        f1 = (sum(o[t] for t in range(1,T+1)))/(T*(1-threshold))
        f2 = (sum(QualCost[j,k]* z_var[j,k] for (j,k) in z_var.keys()))/(T*gamma)
        
        Tau=3
        threshold=0.7
        gamma=4
        Work_Center_ID= range(1,125)
        conts=[]
        
        """
        Adding Constraints (Demand, tau, gamma)
        """
        
        # Demand
        for (j,t) in [(j,t) for j in JobType for t in range(1,T+1) if (j,t) in Demand.keys()]:
            conts.append(Demand[j,t]-sum(x[Id[j,k,t]] for k in FeasibleMachines[j]) ) 
        
        # Tau constraints
        for (j,t) in [(j,t) for j in JobType for t in range(1,T+1) if (j,t) in Demand.keys()]:
            conts.append(sum(s_var[j,k,t] for k in FeasibleMachines[j] if (j,k,t) in xs.keys() )-Tau)
            
        # upper bound on o
        for t in range(1,T+1):
            conts.append(o[t]-1+threshold)
        
        ##############################################################
        # to impose gamma limitation we solve an LP
        ##############################################################
        
        model = grb.Model()
        vv = model.addVars(grb.tuplelist([(j,k,t) for j in JobType for k in FeasibleMachines[j] \
                                          for t in range(1,T+1) if (j,k) in z_var]), vtype=grb.GRB.BINARY,name='vv')
        maxi = model.addVars(grb.tuplelist([t for t in range(1,T+1) ]), vtype=grb.GRB.CONTINUOUS,name='vv')
        model.setObjective(sum(maxi[t] for t in range(1,T+1)),grb.GRB.MINIMIZE)
        for t in range(1,T+1):
            model.addConstr(grb.quicksum(vv[j,k,t] for (j,k) in z_var.keys()  )-gamma <= maxi[t])
        for (j,k) in z_var.keys():
                    for t in range(1,T+1):
                        if (j,k,t) in s_var.keys():
                            model.addConstr(grb.quicksum(vv[j,k,l] for l in range(1,t+1))>= s_var[j,k,t])
        model.setParam("OutputFlag",0)
        model.setParam("timelimit",100)
        model.optimize()
        
        ################################################################
        
        for t in range(1,T+1):
            if maxi[t].x > 0:
                conts.append(maxi[t].x)
            else:
                conts.append(0)
                                    
        out["F"] = [f1, f2]
        out["G"] = conts



In [113]:
ref_dirs = get_reference_directions("das-dennis", 2, n_partitions=12)
ref_dirs[:,0]= ref_dirs[:,0]*2.4
ref_dirs[:,1] = ref_dirs[:,1]*80
problems = MyProblem()
pop = Population.new("X", np.transpose(X))
Evaluator().eval(problems, pop)
termination = get_termination("time", "02:00:00")


algorithm = NSGA3(n_offsprings=20,
                  ref_dirs=ref_dirs, sampling=pop,
    crossover=get_crossover("int_sbx", prob=1.0, eta=3.0),
    mutation=get_mutation("int_pm", eta=3.0),
    eliminate_duplicates=True)

res = minimize(problems,
               algorithm,
               termination,
               seed=1,
               save_history=True,
               verbose=True)

n_gen |  n_eval |   cv (min)   |   cv (avg)   |  n_nds  |     eps      |  indicator  
    1 |       0 |  0.00000E+00 |  0.00000E+00 |       2 |            - |            -
    2 |      20 |  0.00000E+00 |  4.63203E+03 |       2 |  0.00000E+00 |            f
    3 |      40 |  0.00000E+00 |  0.00000E+00 |       2 |  0.00000E+00 |            f
    4 |      60 |  0.00000E+00 |  0.00000E+00 |       2 |  0.00000E+00 |            f
    5 |      80 |  0.00000E+00 |  0.00000E+00 |       2 |  0.00000E+00 |            f
    6 |     100 |  0.00000E+00 |  0.00000E+00 |       2 |  0.00000E+00 |            f
    7 |     120 |  0.00000E+00 |  0.00000E+00 |       2 |  0.00000E+00 |            f
    8 |     140 |  0.00000E+00 |  0.00000E+00 |       2 |  0.00000E+00 |            f
    9 |     160 |  0.00000E+00 |  0.00000E+00 |       2 |  0.00000E+00 |            f
   10 |     180 |  0.00000E+00 |  0.00000E+00 |       2 |  0.00000E+00 |            f
   11 |     200 |  0.00000E+00 |  0.00000E+00 |       

Only two feasible solutions are found

In [114]:
len(res.X)

2